In [5]:
import cv2
import torch
import numpy as np
from PIL import Image


In [6]:
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")

use_large_model = True

if use_large_model:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
else:
    midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

device = "cpu"
midas.to(device)

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if use_large_model:
    transform = midas_transforms.default_transform
else:
    transform = midas_transforms.small_transform


Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/kevinstoesser/.cache/torch/hub/facebookresearch_WSL-Images_main
Using cache found in /Users/kevinstoesser/.cache/torch/hub/intel-isl_MiDaS_master


In [7]:
def depth(img):
    cv_image = np.array(img)
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img


In [12]:
test_img = Image.open('/Users/kevinstoesser/Pictures/Background/DSCF1321.jpg')

test_depth = depth(test_img)
test_depth.show()
test_depth = np.array(test_depth)
print(test_depth)

[[135 135 135 ... 118 118 118]
 [135 135 135 ... 118 118 118]
 [135 135 135 ... 118 118 118]
 ...
 [207 207 207 ... 255 254 254]
 [207 207 207 ... 254 254 254]
 [207 207 207 ... 254 254 254]]


In [13]:
# Load the grayscale image and depth map
image = cv2.imread('/Users/kevinstoesser/Pictures/Background/DSCF1321.jpg')
depth_map = test_depth

# Normalize the depth map so that it is in the range [0, 1]
depth_map = depth_map/ 255.0

# Set the depth at which no blur should be applied, and the range of depths over which the blur strength increases
no_blur_depth = 0.5
blur_range = 0.5

# Create an empty output image with the same size as the input image
output = np.zeros_like(image)

# Loop over all pixels in the image
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        # Get the depth value for the current pixel
        depth = depth_map[i, j]

        # Calculate the kernel size for the Gaussian blur based on the depth value
        if depth > no_blur_depth + blur_range:
            kernel_size = 5
        elif depth > no_blur_depth:
            kernel_size = 5 + int((depth - no_blur_depth) / blur_range * 20)
        else:
            kernel_size = 0
        if kernel_size % 2 == 0:
            kernel_size += 1

        # Apply Gaussian blur to the current pixel with the calculated kernel size
        if kernel_size > 0:
            output[i, j] = cv2.GaussianBlur(
                image[i, j], (kernel_size, kernel_size), 0)[0][0]
        else:
            output[i, j] = image[i, j]

# Save the output image
cv2.imwrite('output.jpg', output)


KeyboardInterrupt: 